In [2]:
import fingertips_py as ftp
import pandas as pd
import requests

I need to know for each indicator, which area types and year conifgurations exist so that I know which ones I can pick from and minimise computation for data extraction. - Need to see if I can get a list of this

The alternative would be to create a hierarchy of preference and iterate through the data for each hierachy level. If an indicator is found within a level, then it will no longer be queried for further levels. This might still be computationally expensive but needs to be investigated further 

# Area types available for each indicator

In [3]:
raw_available_indicator_at_area = pd.read_json('https://fingertips.phe.org.uk/api/available_data').fillna(value='null')

print(raw_available_indicator_at_area['AreaTypeId'].unique()
      ,len(raw_available_indicator_at_area['AreaTypeId'].unique()),
      len(raw_available_indicator_at_area),
      raw_available_indicator_at_area['IndicatorId'].nunique()
    #   ,
    #   available_indicator_at_area['IndicatorId'].unique()
)

[402 401 102 101  15   6 221 220 204 167 166  66   7 302 219 152 301 223
 202 165 154 201  46  10 126 104 119 203 118 205 112 129 222   8   3  41] 36 8910 1693


In [4]:
raw_available_indicator_at_area.columns

Index(['IndicatorId', 'AreaTypeId'], dtype='object')

# Indicator metadata

In [5]:
indicator_metadata = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/csv/all')

In [6]:
indicator_metadata['Indicator ID'].nunique()

1403

# Area Ids mapped to descriptive data

In [7]:
area_ref = pd.read_json('https://fingertips.phe.org.uk/api/area_types').fillna(value='null')

In [8]:
area_ref.columns

Index(['Id', 'Name', 'Short', 'Class', 'Sequence', 'CanBeDisplayedOnMap'], dtype='object')

# Area types available for each indicator with extra area id reference data

In [9]:
available_indicator_at_area = pd.merge(left=raw_available_indicator_at_area, right=area_ref,
         how='left', left_on='AreaTypeId', right_on='Id',
         indicator=True).rename({'_merge': 'area_merge'}, axis=1)

available_indicator_at_area = pd.merge(left=available_indicator_at_area, right=indicator_metadata[['Indicator ID', 'Indicator']],
                                       left_on='IndicatorId', right_on='Indicator ID', how='inner',
                                       indicator=True, )

In [10]:
available_indicator_at_area

,IndicatorId,AreaTypeId,Id,Name,Short,Class,Sequence,CanBeDisplayedOnMap,area_merge,Indicator ID,Indicator,_merge
0,108,402,402,Upper tier local authorities (post 4/21),Counties & UAs (from Apr 2021),ua-county-composite,2021,True,both,108,Under 75 mortality rate from all causes,both
1,108,401,401,Lower tier local authorities (post 4/21),Districts & UAs (from Apr 2021),ua-la-composite,2021,True,both,108,Under 75 mortality rate from all causes,both
2,108,102,102,Upper tier local authorities (pre 4/19),Counties & UAs (pre Apr 2019),ua-county-composite,2018,True,both,108,Under 75 mortality rate from all causes,both
3,108,101,101,Lower tier local authorities (pre 4/19),Districts & UAs (pre Apr 2019),ua-la-composite,2018,True,both,108,Under 75 mortality rate from all causes,both
4,108,15,15,England,England,null,0,False,both,108,Under 75 mortality rate from all causes,both
...,...,...,...,...,...,...,...,...,...,...,...,...
8059,93966,7,7,General Practice,GPs,null,0,True,both,93966,Number of clinical workforce (FTE) in general ...,both
8060,93967,402,402,Upper tier local authorities (post 4/21),Counties & UAs (from Apr 2021),ua-county-composite,2021,True,both,93967,Mortality rate from dementia and Alzheimer's d...,both
8061,93967,401,401,Lower tier local authorities (post 4/21),Districts & UAs (from Apr 2021),ua-la-composite,2021,True,both,93967,Mortality rate from dementia and Alzheimer's d...,both
8062,93967,15,15,England,England,null,0,False,both,93967,Mortality rate from dementia and Alzheimer's d...,both


In [11]:
available_indicator_at_area['_merge'].unique()

['both']
Categories (3, object): ['left_only', 'right_only', 'both']

In [12]:
available_indicator_at_area[available_indicator_at_area['_merge']=='left_only']

,IndicatorId,AreaTypeId,Id,Name,Short,Class,Sequence,CanBeDisplayedOnMap,area_merge,Indicator ID,Indicator,_merge


In [13]:
available_indicator_at_area.columns

Index(['IndicatorId', 'AreaTypeId', 'Id', 'Name', 'Short', 'Class', 'Sequence',
       'CanBeDisplayedOnMap', 'area_merge', 'Indicator ID', 'Indicator',
       '_merge'],
      dtype='object')

In [14]:
# available_indicator_at_area.to_csv('available_indicator_at_area.csv')

In [15]:
indicators_per_area = (pd.DataFrame(available_indicator_at_area.groupby(['Name', 'Short', 'Class', 'Sequence','CanBeDisplayedOnMap'])['_merge'].count())
                       .reset_index()
                       .sort_values(['Class', 'Sequence'], ascending=False)
                       .rename({'_merge': 'indicator_count'}, axis=1)
                       )

In [16]:
# if __name__ == "__main__":
#     indicators_per_area.to_csv('indicators_per_area.csv')

In [17]:
indicator_at_area_pivot = available_indicator_at_area[['Indicator', 'Name', 'Id']].pivot_table(index='Indicator', columns='Name', aggfunc='count')

In [18]:
# indicator_at_area_pivot.to_csv('indicator_at_area_pivot.csv')

# Retrieve indicator data

In [19]:
# # metadata_areas = ftp.metadata.get_all_areas(is_test=False)
# # metadata_value_notes = ftp.metadata.get_all_value_notes(is_test=False)
# # metadata_profiles = ftp.metadata.get_all_profiles(is_test=False)
# # metadata_domains = ftp.metadata.get_domains_in_profile(18)
# metadata_all_indicators = ftp.metadata.get_metadata_for_all_indicators_from_csv(is_test=False)
# metadata_all_indicator_ids = metadata_all_indicators['Indicator ID'].tolist()

In [20]:
# metadata_all = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/all?include_definition=yes&include_system_content=yes')

In [21]:
# replace_comma = '%2C'
# ids_as_str = replace_comma.join([str(i) for i in metadata_all_indicator_ids])

In [22]:
# data = ftp.retrieve_data.get_all_data_for_indicators(indicators=[108,
#  113,
#  114,
#  200,
#  212],
#                                                                area_type_id=7,
#                                                                parent_area_type_id=15,
#                                                                filter_by_area_codes=None,
#                                                                is_test=False)

In [23]:
# ids_as_str

In [24]:
# data = ftp.retrieve_data.get_data_by_indicator_ids(indicator_ids=ids_as_str, # [Maximum 100]
#                                                    area_type_id=7,
#                                                    parent_area_type_id=15,
#                                                    profile_id=None,
#                                                    include_sortable_time_periods=None,
#                                                    is_test=False)